In [207]:
def remove_outliers(df, col):

    mean_bound = df[col].mean()
    outliers_mask = (df[col] < mean_bound)

    df = df[~outliers_mask]

    return df

In [208]:
def plot_death_by_causes(df, title, x='causes', y='quantity'):
    fig = px.bar(df, x=x, y=y)

    fig.update_layout(
        xaxis=dict(categoryorder='total descending'),
        title=title
    )
    fig.show()

### Quais são os municípios com as maiores taxas de óbitos por causas evitáveis em crianças de 0 a 5 anos?

In [209]:
df_childs = pd.read_csv('./data/born_deaths_region.csv', index_col=0)
df_childs = df_childs.sum(axis=1).reset_index(name='city').rename(columns={'city': 'quantity', 'index': 'city'})

In [210]:
df_childs = remove_outliers(df_childs, 'quantity')

In [211]:
plot_death_by_causes(df_childs, "Mortes por município (crianças de 0-5 anos)", x='city')

In [ ]:
# Normalize values

min_value = df_childs['quantity'].min()
max_value = df_childs['quantity'].max()

df_childs['score'] = (df_childs['quantity'] - min_value) / (max_value - min_value)

In [ ]:
babies_map = get_pe_map(df_childs)
babies_map

### Existe uma correlação entre o número de consultas pré-natal realizadas e o peso ao nascer dos bebês em determinado grupo de municípios?

In [214]:
df_babies_weight = pd.read_csv('./data/babies_weight.csv')

In [215]:
df_babies_weight['peso ao nascer'] = df_babies_weight['peso ao nascer'].str.split('-').map(lambda x: median([int(x[0]), int(x[1])]))

In [216]:
df_babies_weight

,peso ao nascer,nenhuma,1-3 vezes,4-6 vezes,7 e +,não informado,ignorado
0,300.0,6,33,48,87,0,4
1,749.5,23,185,305,95,0,21
2,1249.5,43,162,494,295,1,23
3,1999.5,211,813,2797,5338,4,109
4,2749.5,407,1475,5920,20305,11,228
5,3499.5,804,3349,14188,64547,48,536
6,4499.5,75,282,1113,6053,1,30


In [217]:
# Select the columns representing the values of medical consultations
consultation_columns = ['peso ao nascer', 'nenhuma', '1-3 vezes', '4-6 vezes', '7 e +', 'não informado', 'ignorado']

# Create a subset DataFrame with only the birth weight and medical consultation columns
subset_df = df_babies_weight[consultation_columns]

row_index = 'peso ao nascer'

# Calculate the correlation matrix
correlation = subset_df.corr().loc[row_index]

# Create a bar plot of the correlation values using Plotly
fig = go.Figure(data=[go.Bar(x=correlation.index, y=correlation.values)])

# Customize the layout of the plot
fig.update_layout(
    title=f'Correlation Plot of "{row_index}" with Medical Consultations',
    xaxis=dict(title='Medical Consultation Categories', tickangle=45),
    yaxis=dict(title='Correlation')
)

# Show the plot
fig.show()

### Quais são as principais causas de óbitos evitáveis em diferentes faixas etárias nos municípios com maiores taxas de mortalidade?

##### Crianças de 0-5 anos

In [218]:
df_causes_born = pd.read_csv('./data/born_deaths_causes.csv', index_col=0)
df_causes_born = df_causes_born.sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})

In [219]:
df_causes_born = remove_outliers(df_causes_born, 'quantity')

In [220]:
plot_death_by_causes(df_causes_born, "Mortes por causas evitáveis (crianças de 0-5 anos)")

##### Diferentes faixas

In [221]:
df_causes_adulthood = pd.read_csv('./data/adulthood_deaths_causes.csv', index_col=0)

In [222]:
df_child = df_causes_adulthood[['5 a 9 anos', '10 a 14 anos']].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})
df_young = df_causes_adulthood[['15 a 19 anos']].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})
df_adult = df_causes_adulthood[['20 a 29 anos', '30 a 39 anos', '40 a 49 anos', '50 a 59 anos']].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})
df_elderly = df_causes_adulthood[['60 a 69 anos', '70 a 74 anos']].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})

In [223]:
df_child = remove_outliers(df_child, 'quantity')
df_young = remove_outliers(df_young, 'quantity')
df_adult = remove_outliers(df_adult, 'quantity')
df_elderly = remove_outliers(df_elderly, 'quantity')

In [224]:
plot_death_by_causes(df_child, "Mortes por causas evitáveis (5-14 anos)")

In [225]:
plot_death_by_causes(df_young, "Mortes por causas evitáveis (15-19 anos)")

In [226]:
plot_death_by_causes(df_adult, "Mortes por causas evitáveis (20-59 anos)")

In [227]:
plot_death_by_causes(df_elderly, "Mortes por causas evitáveis (60-74 anos)")

### Quais são os municípios com os melhores indicadores de saúde básica, considerando a taxa de óbitos evitáveis e a disponibilidade de serviços básicos de saúde?

In [239]:
df_places = pd.read_csv('./data/health_places.csv', index_col=0)

In [240]:
df_deaths_city = pd.read_csv('./data/deaths_by_city.csv', index_col=0)
df_deaths_city = df_deaths_city.T

df_child = df_deaths_city[['5 a 9 anos', '10 a 14 anos']].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'city'})
df_young = df_deaths_city[['15 a 19 anos']].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'city'})
df_adult = df_deaths_city[['20 a 29 anos', '30 a 39 anos', '40 a 49 anos', '50 a 59 anos']].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'city'})
df_elderly = df_deaths_city[['60 a 69 anos', '70 a 74 anos']].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'city'})

In [231]:
df_places = df_places.sum(axis=1).reset_index(name='quantity_ubs').sort_values(by='quantity_ubs', ascending=False)

In [232]:
df_merged_child = pd.merge(df_child, df_places, on='city', how='right')
df_merged_young = pd.merge(df_young, df_places, on='city', how='right')
df_merged_adult = pd.merge(df_adult, df_places, on='city', how='right')
df_merged_elderly = pd.merge(df_elderly, df_places, on='city', how='right')

In [243]:
df_population = pd.read_csv('./data/pe_population.csv')
df_population.head()

,city,population
0,abreu e lima,100699
1,afogados da ingazeira,37545
2,afranio,19978
3,agrestina,25240
4,agua preta,37385


In [244]:
pd.merge(df_population, df_merged_child, on='city', how='left')

,city,population,quantity,quantity_ubs
0,abreu e lima,100699,2.0,30.0
1,afogados da ingazeira,37545,1.0,16.0
2,afranio,19978,1.0,13.0
3,agrestina,25240,0.0,18.0
4,agua preta,37385,1.0,16.0
...,...,...,...,...
180,vertente do lerio,7529,0.0,5.0
181,vertentes,21170,0.0,6.0
182,vicencia,32899,2.0,14.0
183,vitoria de santo antao,140388,8.0,40.0
